In [1]:
from FrozenYoghourt import *
from FrozenYoghourt.mode import *
from FrozenYoghourt.gates import *
from FrozenYoghourt.maths import *
from FrozenYoghourt.quantum import *

In [2]:
import matplotlib.pyplot as plt

In [3]:
def double_cosets(g, h):
     E = Magic()

     g, h = to_su(g), to_su(h)

     u, v = np.conj(E).T @ g @ E, np.conj(E).T @ h @ E

     D1, P1 = np.linalg.eig(u @ u.T)
     D2, P2 = np.linalg.eig(v @ v.T)

     idx = D1.argsort()[::-1]
     D1 = D1[idx]
     P1 = P1[:, idx]

     idx = D2.argsort()[::-1]
     D2 = D2[idx]
     P2 = P2[:, idx]

     if not np.isclose(D1[0], D2[0]):
         D2 = -D2
         idx = D2.argsort()[::-1]
         D2 = D2[idx]
         P2 = P2[:, idx]

     a = P1.T
     b = P2.T

     c = np.conj(v).T @ b.T @ a @ u

     left_coset = E @ b.T @ a @ np.conj(E).T
     right_coset = E @ c.T @ np.conj(E).T

     return left_coset, right_coset

In [4]:
global_phase??

In [5]:
def KAK(U:np.ndarray):
    
    M = Magic()
    
    phase1 = global_phase(to_su(U), U)
    

    return K2, A, K1, phase

In [6]:
def canonical_class_vector(U, return_phase = True):
    
    M = Magic()
    V = dagger(M)@to_su(U)@M
    D_squared, P = np.linalg.eig(V.T@V)

    spectrum = np.angle(D_squared) # These steps make sure that D stays unimodular after the square root
    spectrum[3] -= np.sum(spectrum)
    spectrum = (spectrum/2).reshape(4, )

    tx, ty, tz = (Gamma().T@spectrum)[1:]
    
    # Step 1 and 2
    param = np.array([tx, ty, tz])
    shift = param // np.pi
    phase = np.exp(-1j*(np.pi/2)*(np.sum(shift)%2)) # Keep track of whether to add phase to correct for KAK
    k = np.sort(param - np.pi*shift)[::-1]

    # Step 3
    if k[0] + k[1] > np.pi:
        k = np.sort(np.array([np.pi - k[1], np.pi - k[0], k[2]]))[::-1]

    # Step 4
    if np.isclose(k[2], 0) and (k[0] > np.pi/2):
        k[0] = np.pi - k[0]
        k = np.sort(k)[::-1]
        phase *= 1j
    
    if return_phase:
        return k[0], k[1], k[2], phase
    else:
        return k[0], k[1], k[2]

In [7]:
def is_local(U):
    M = Magic()
    V = dagger(M)@to_su(U)@M
    if np.isclose(np.linalg.det(V), 1) and np.all(np.isclose(V@V.T, np.identity(4))):
        return True
    else:
        return False

In [8]:
U = to_su(random_unitary(4).data)

In [75]:
def KAK(U:np.ndarray):
    
    phase1 = global_phase(to_su(U), U)
    tx, ty, tz, phase2 = canonical_class_vector(U)
    
    total_phase = phase1*phase2
    Can = CAN(tx, ty, tz)
    
    L, R = double_cosets(phase2*Can, to_su(-1j*U))

    return L, Can, R, total_phase

In [77]:
U

array([[-0.27512678+0.14805447j, -0.23023844+0.22473805j,
         0.34361378+0.68223311j, -0.00671838+0.46401575j],
       [ 0.09279758+0.52471081j,  0.07985559+0.43534232j,
        -0.42429982+0.30854319j,  0.2629816 -0.41925979j],
       [-0.10731208-0.29925107j, -0.08676609+0.79631563j,
         0.07561382-0.23857059j, -0.42769583-0.10829999j],
       [ 0.44580103+0.56448796j, -0.2426256 +0.00831291j,
        -0.09920046-0.26226745j, -0.41789457+0.41281676j]])

In [79]:
U = to_su(random_unitary(4, seed = 1).data)

L, A, R, phase = KAK(U)

#aae(phase*L@A@R, 1j*U)

is_local(L)

False

In [10]:
for _ in range(100):
    U = to_su(random_unitary(4).data)

    L, A, R, phase = KAK(U)

    aae(phase*L@A@R, U)

    assert is_local(L), 'L is not local'
    assert is_local(R), 'R is not local'

AssertionError: L is not local

In [235]:
is_local(L)

False

In [237]:
is_local(R)

False

In [1]:
f = open("CHANGELOG.md", "r")

In [2]:
print(f.read())

# Change Log

0.0.1 (10/02/2022)
------------------

1. Wrote README file
2. Add P gates method to gates.py
3. Add CU method to gates.py
4. Add view method to mode.py
5. Add log.txt for keeping track of changes

0.0.3 (11/02/2022)
------------------

1. Move to_su to maths.py
2. Move kron_decomp to quantum.py
3. Change default variable in the chi method to "x"

0.0.7 (12/02/2022)
------------------

1. Import gates.py to quantum.py

0.0.8 (12/02/2022)
------------------

1. Fix Quantum.double_cosets by importing the correct packages
2. Add default_import method to allow for faster import prompt
3. Change random_local_gates to random_local_ops and allow for creating more operation at the same time.
4. Allow for doing to_su on list of matrices.

0.0.10.1 (15/02/2022)
-------------------

1. Add view method to visualize numerical matrices
2. Add CAN method
3. Add Gamma gates

0.0.11 (17/02/2022)
-------------------

1. Delete Class from files so now g.CAN will just be CAN. Although the Mo